<a href="https://colab.research.google.com/github/Noahbrauer/Competition-Part-3/blob/master/ML_Comp_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble  import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

X_train = np.loadtxt('train_features.csv', skiprows=1, delimiter=',')
X_test = np.loadtxt('test_features.csv', skiprows=1, delimiter=',')
y_train = np.loadtxt('train_values.csv', skiprows=1, delimiter=',').astype('int32')

#Trying to find a better way to classify day of year as a cyclical value
def cyclical(inFile):
  df = pd.read_csv(inFile,  ',', index_col=0)
  df1 = pd.to_datetime(df[['day', 'month', 'year']])
  dfTemp = df1.dt.dayofyear
  temp1 = np.sin(2*np.pi*dfTemp/366)
  temp2 = np.cos(2*np.pi*dfTemp/366)
  d = {'sin_date': temp1, 'cos_date': temp2,  'date': df1}
  df1 = pd.DataFrame(data=d)
  df1 = df1[['sin_date', 'cos_date', 'date']]
  result = pd.concat([df[['decimalLatitude','decimalLongitude']], df1], axis=1)
  result_train = (result.sort_values(by='date')).to_numpy()[:,:-1]
  return result_train

#Handle ID column
X_train = cyclical('train_features.csv')
y_train = y_train[:,1:]
X_test_labels = X_test[:,(0,)]
X_test = cyclical('test_features.csv')

#Scaling all the values
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_new_train, X_new_test, y_new_train, y_new_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [ ]:
clf = RandomForestClassifier(bootstrap=True, max_depth=30,max_features='sqrt', min_impurity_decrease=0.0, min_samples_leaf=1, min_samples_split=5, min_weight_fraction_leaf=0.0, n_estimators=1400,verbose=0)
clf.fit(X_new_train, y_new_train)
lab_enc = preprocessing.LabelEncoder()
predict = clf.predict(X_new_train)
print(predict)
print(lab_enc.fit_transform(y_new_train))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


[8 3 4 ... 6 0 7]
[8 8 9 ... 1 0 7]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:

from sklearn.metrics import accuracy_score
print(predict)
print(lab_enc.fit_transform(y_new_train))
score = accuracy_score(predict,lab_enc.fit_transform(y_new_train))
print(score)

print(acc)
print(X_new_test)
print(y_new_test)acc = clf.score(X
predw_test,predictw_test)

[8 3 4 ... 6 0 7]
[8 8 9 ... 1 0 7]
0.3823238239481894


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# This was the code i used for testing, I deleted it but readded it for submission

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 1500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(train_features, train_labels)